In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Running in Colab: download project-local imports.
    !wget -q https://raw.githubusercontent.com/maxpoletto/it-emissions/main/input_table.py
    import sys
    sys.path.append('.')
    print("Setup complete for Colab environment.")


In [ ]:
from input_table import create_inputs
from ipywidgets import HBox, Layout, Output, Textarea
import matplotlib.pyplot as plt
import numpy as np

energy_mix = {
    "switzerland": {
        # From app.electricitymaps.org
        "g_per_kwh": 54
        # https://www.bafu.admin.ch/bafu/de/home/themen/klima/klimawandel--fragen-und-antworten.html
        # has a substantially higher (2x) value.
    },
    "eu27": {
        # From https://www.eea.europa.eu/en/analysis/indicators/greenhouse-gas-emission-intensity-of-1/greenhouse-gas-emission-intensity-of-electricity-generation-country-level        
        "g_per_kwh": 210
    }
}
hardware = {
    "laptop": {
        "lifetime_y": 4,
        "count": 10175,
        "prod_co2e_kg": 221,
        "disp_co2e_kg": -4, # Ecoinvent has negative value??
        "power_w": 50,
        "power_idle_w": 1,
        "run_mins_per_day": 60*6,
        "run_days_per_year": 5*52,
    },
    "desktop-pc": { # Hörsaalcomputer
        "lifetime_y": 6,
        "count": 150,
        "prod_co2e_kg": 322,
        "disp_co2e_kg": -19, # Ecoinvent has negative value??
        "power_w": 100,
        "power_idle_w": 1,
        "run_mins_per_day": 60*2,
        "run_days_per_year": 5*35,
    },
    "desktop-monitor": {
        "lifetime_y": 6,
        "count": 10175 + 150, # 10175 laptops + 150 desktop-pc
        "prod_co2e_kg": 342,
        "disp_co2e_kg": 2,
        "power_w": 30,
        "power_idle_w": 1,
        "run_mins_per_day": 60*3,
        "run_days_per_year": 5*45, # some PC, some laptop
    },
    "server": {
        "lifetime_y": 5,
        "count": 1028,
        "prod_co2e_kg": 400,
        "disp_co2e_kg": 5, # Not in Ecoinvent, just a guess
        "power_w": 300,
        "power_idle_w": 20,
        "run_mins_per_day": 60*24,
        "run_days_per_year": 365,
    },
    "printer-scanner-copier": {
        "lifetime_y": 6,
        "count": 333,
        "prod_co2e_kg": 1167,
        "disp_co2e_kg": 45,
        "power_w": 700,
        "power_idle_w": 30,
        "run_mins_per_day": 10,
        "run_days_per_year": 5*52,
    },
    "projector": {
        "lifetime_y": 6,
        "count": 20,
        "prod_co2e_kg": 30,
        "disp_co2e_kg": 50,
        "power_w": 70,
        "power_idle_w": 80,
        "run_mins_per_day": 90,
        "run_days_per_year": 100,
    },
    "smartphone": {
        "lifetime_y": 3,
        "count": 2389,
        "prod_co2e_kg": 38,
        "disp_co2e_kg": 0, # Another strange Ecoinvent value
        "power_w": 5,
        "power_idle_w": 1,
        "run_mins_per_day": 60*16,
        "run_days_per_year": 365,
    },
}

ml = {
    "llm use": {
        "num_users": 12000,
        "queries_per_day": 20,
        "days_per_year": 300,
        "wh_per_query": 3,
    },
}

# Function to update the bar chart
def recalc_hardware(hardware):
    emissions = {}
    for device, params in hardware.items():
        wh = (params["power_w"] * params["run_mins_per_day"] * params["run_days_per_year"]
              + params["power_idle_w"] * (24*60 - params["run_mins_per_day"]) * params["run_days_per_year"]
              + params["power_idle_w"] * (24*60) * (365 - params["run_days_per_year"])) / 60
        kwh = wh / 1000
        kgco2 = kwh * energy_mix["switzerland"]["g_per_kwh"] / 1000
        kgco2 += (params["prod_co2e_kg"] + params["disp_co2e_kg"]) / params["lifetime_y"]
        emissions[device] = kgco2 * params["count"]
    return emissions

def recalc_ml(ml):
    emissions = {}
    for type, params in ml.items():
        kwh = params["num_users"] * params["queries_per_day"] * params["days_per_year"] * params["wh_per_query"] / 1000
        emissions[type] = kwh * energy_mix["eu27"]["g_per_kwh"] / 1000
    return emissions

def monte_carlo_emissions(hardware, ml, iterations=3):
    emissions = {key: [] for key in {**hardware, **ml}}
    variations = {
        "hardware": {
            "power_w": 0.1,  # ±10% variation
            "run_mins_per_day": 0.2,  # ±20% variation
            "lifetime_y": 0.15,  # ±15% variation
        },
        "ml": {
            "queries_per_day": 0.25,  # ±25% variation
            "wh_per_query": 0.3,  # ±30% variation
        }
    }
    
    for _ in range(iterations):
        hw_copy = {k: v.copy() for k, v in hardware.items()}
        ml_copy = {k: v.copy() for k, v in ml.items()}

        for _, params in hw_copy.items():
            for param, var_pct in variations["hardware"].items():
                params[param] *= (1 + np.random.normal(0, var_pct))
        for _, params in ml_copy.items():
            for param, var_pct in variations["ml"].items():
                params[param] *= (1 + np.random.normal(0, var_pct))        
        
        # Append results to emissions
        res = {**recalc_hardware(hw_copy) , **recalc_ml(ml_copy)}
        for key, value in emissions.items():
            value.append(res[key])
    return emissions

def plot_monte_carlo_results(emissions):
    print(emissions)
    with output2:
        output2.clear_output(wait=True)
        plt.figure(figsize=(10, 8))

        # Boxplot for each key in emissions.
        keys = list(emissions.keys())
        values = [emissions[key] for key in keys]
        plt.boxplot(values, widths=0.6, showfliers=False, positions=range(len(keys)), tick_labels=keys)

        plt.title("Emissions by category: Monte Carlo Simulation")
        plt.ylabel("kg CO2e")
        plt.xticks(rotation=45, ha="right", fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout()
        plt.show()

def update_charts(_):
    hardware_emissions = recalc_hardware(hardware)
    ml_emissions = recalc_ml(ml)
    with output:
        output.clear_output(wait=True)
        plt.figure(figsize=(10, 8))
        all_emissions = {**hardware_emissions, **ml_emissions}
        plt.bar(all_emissions.keys(), all_emissions.values())
        plt.title("Emissions by category", fontsize=12)
        plt.ylabel("kg CO2e", fontsize=10)
        plt.xticks(rotation=45, ha="right", fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout()
        plt.show()
    infotext.value = f"Hardware emissions: {sum(hardware_emissions.values()):.0f} kg CO2e\nML emissions: {sum(ml_emissions.values()):.0f} kg CO2e"
    mce = monte_carlo_emissions(hardware, ml)
    plot_monte_carlo_results(mce)

inputs0 = create_inputs(energy_mix, update_charts, column_major=True)
inputs1 = create_inputs(hardware, update_charts)
inputs2 = create_inputs(ml, update_charts)
output = Output()
output2 = Output()
outbox = HBox([output, output2]) # Maybe add more outputs later
infotext = Textarea(value='', layout=Layout(width='99%', height='40px', overflow='hidden'))
display(inputs0, inputs1, inputs2, outbox, infotext, clear=True)
update_charts(_)
